In [ ]:
class Arr:
    def __init__(self, data):
        data = [item for item in data]
        self.dtype = dtype(data)
        self.data = [self.dtype(item) for item in data]
        self.size = len(self.data) # Because len will be something else
        self.shape = (self.size,)
        self.ndim = len(self.shape)

    def __str__(self):
        ret = "Arr: \n" # We will add more strings to this "main" string
        if self.ndim < 2: 
            ret += str(self.data)
        else:
            rows, cols = self.shape
            for i, val in enumerate(self.data):
                ret += str(val).center(6) # str.center - check it out.
                if i % cols == cols -1: # in "real" math this means i mod cols = -1
                    ret += '\n'
    
        return ret

    __repr__ = __str__

    def reshape(self, rows, cols=None):
        ret = Arr(self.data)
        if cols is None:
            ret = ret.reshape(rows, 1)
            ret.shape = (ret.shape[0],)
            ret.ndim = len(ret.shape)
            return ret
        elif cols == -1:
            assert self.size % rows == 0, f"rows must be divide {self.size} without a remainder"
            cols = self.size // rows
        elif rows == -1:
            assert self.size % cols == 0, f"cols must be divide {self.size} without a remainder"
            rows = self.size // cols
        assert rows * cols == len(self.data), (f"cannot reshape data with {self.size}"
                                               f" values to shape ({rows},{cols})")
        ret.shape = (rows, cols)
        ret.ndim = len(ret.shape)
        return ret

    def __getitem__(self, items):
        return arr_getter(self, items)
    
    def __len__(self):
        return self.shape[0]

    def __iter__(self):
        return iter(self.data)

    ### Math
    def sum(self):
        return sum(self)

    def count(self):
        return len(self)

    def mean(self):
        return (self.sum() / self.count())
    
    ### Arithmetic

    def __add__(self, other):
        return zip_apply(self, other, self.dtype.__add__)

    def __sub__(self, other):
        return zip_apply(self, other, self.dtype.__sub__)

    def __mul__(self, other):
        return zip_apply(self, other, self.dtype.__mul__)

    def __truediv__(self, other):
        return zip_apply(self, other, self.dtype.__truediv__)

    def __abs__(self):
        return Arr(map(self.dtype.__abs__, self.data))


def zip_apply(left, right, f):
    # Length is the same
    assert len(left) == len(right), f'arrays are not of same shape'
    # Type is the same
    assert dtype(left) == dtype(right), f'Arrays are not of same dtype'
    # We can do the work
    result = [f(l, r) for (l, r) in zip(left, right)]
    return Arr(result)


def dtype(obj):
    """Returns the dtype of the array"""
    dtype = int
    for item in obj:
        itype = type(item)
        if itype == str:  # str is the largest, so dtype is str
            return str
        if itype == float:  # We haven't seen str by now so type is either float or int
            dtype = float
    return dtype

def indices2len(start, stop, step):
    if step > 0:
        return (stop - start - 1) // step + 1
    elif step == 0:
        return 1
    else:
        # No reversing
        raise ValueError
        

def coord2idx(arr, row, col):
    return row * arr.shape[1] + col

def idx2coord(arr, idx):
    row = idx // arr.shape[1]
    col = idx % arr.shape[0]
    return row, col

def arr_getter(arr, items):
    if not isinstance(items, tuple):  # got only one slicer
        items = (items,)
    assert arr.ndim >= len(items), f'More slicers ({len(items)}) than dimensions ({arr.ndim})'
    if arr.ndim == 1:
        return arr.data[items[0]]
    if arr.ndim == 2:
        # Less Then 2 Items
        if len(items) < 2:
            ret = arr_getter(arr, items + (slice(None),))
            return ret
        # Int location
        new_items = []
        for item in items:
            if isinstance(item, int):
                new_items.append(slice(item, item+1))
            else:
                new_items.append(item)
        items = tuple(new_items)
        r_start, r_stop, r_step = items[0].indices(arr.shape[0])
        c_start, c_stop, c_step = items[1].indices(arr.shape[1])
        new_shape = (indices2len(r_start, r_stop, r_step), indices2len(c_start, c_stop, c_step))
        return array_iterator(arr, new_shape, r_start, r_step, r_stop, c_step, c_stop, c_start)
    
def array_iterator(arr, new_shape, r_start, r_step, r_stop, c_step, c_stop, c_start):
    r_index, c_index = r_start, c_start # index is set to start
    ret = [] # We will populate this
    for _ in range(new_shape[0] * new_shape[1]):
        if c_index >= c_stop:
            # We reached end of line
            c_index = c_start # Go back to start of line which is first coloumn
            r_index += r_step # Advance the row
        if r_index >= r_stop:
            # We reached the end
            break
        _index = coord2idx(arr, r_index, c_index) # Which index to take from the data
        try:
            ret.append(arr.data[_index])
        except IndexError:
            break
        c_index += c_step
    ret = Arr(ret).reshape(*new_shape)
    return ret